# Export Simulation Results

In [25]:
## Load the BPTK Package
from BPTK_Py.bptk import bptk 

bptk = bptk()

In [14]:
import pandas as pd
import numpy as numpy

In [3]:
def export_scenarios(scenario_manager, scenarios=[], equations, interactive_scenario=None, interactive_settings=[],time_column_name="time", scenario_column_name="scenario",filename):
    manager=bptk.scenario_manager_factory.get_scenario_managers(scenario_managers_to_filter=[scenario_manager])[scenario_manager]
    
    # if no scenarios are passed we export all scenarios
    if scenarios==[]:
        scenarios = manager.get_scenario_names()
    # collect the scenarios, add a new column to include the scenario name and a new column for the time time dimension  
    scenario_dfs=[]
    for scenario in scenarios:
        df = bptk.plot_scenarios(scenario_managers=[scenario_manager],scenarios=[scenario],equations=equations,return_df=True)
        df[scenario_column_name]=[scenario]*len(df.index)
        df[time_column_name]=df.index
        scenario_dfs += [df]
    scenarios_tab = pd.concat(scenario_dfs, ignore_index=True, sort=False) 
    scenarios_tab.to_excel(filename, sheet_name='scenarios')
    # now generate the data for the interactive scenarios
    interactive_dfs=[]
    # generate all combinations of the settings
    settings=[]
    setting_index=0
    for setting in interactive_settings:
        for n in range(*setting["range"]):
            settings[setting_index][setting["name"]]=n
    
    
    

In [4]:
df=export_scenarios("smCustomerAcquisition",["customers","profit"])

In [5]:
df.to_excel('/Users/olivergrasl/Desktop/test.xlsx', sheet_name='scenarios')

In [28]:
settings=[
    {
        "advertisingSuccessPct":adv,
        "referralFreeMonths":months,
        "referralProgramAdoptionPct":adoption,
        "referrals":referals
    }
    for adv in numpy.arange(*(0,0.2,0.01))
    for adoption in numpy.arange(*(0,40,10))
    for months in numpy.arange(*(0,12,1))
    for referals in numpy.arange(*(0,12,1))
]

In [26]:
bptk.add_scenario(dictionary={
                      "smCustomerAcquisition":  
                      {
                          "source":"simulation_models/customer_acquisition.stmx",
                          "model":"simulation_models/customer_acquisition",
                          "interactiveScenario":{
                              "constants":{
                                 "referrals":0,
                                  "advertisingSuccessPct":0.1,
                                  "referralFreeMonths":3,
                                  "referralProgamAdoptionPct":10
                                }
                          }
                    }
                    }
)

In [32]:
interactive_dfs=[]
for setting in settings:
    scenario= bptk.scenario_manager_factory.get_scenarios(["smCustomerAcquisition"],
                                                                          ["interactiveScenario"])["interactiveScenario"]
    scenario.constants["advertisingSuccessPct"]=setting["advertisingSuccessPct"]
    scenario.constants["referralFreeMonths"]=setting["referralFreeMonths"]
    scenario.constants["referralProgramAdoptionPct"]=setting["referralProgramAdoptionPct"]
    scenario.constants["referrals"]=setting["referrals"]
    bptk.reset_simulation_model(scenario_manager="smCustomerAcquisition",
                                                             scenario="interactiveScenario")
    df=bptk.plot_scenarios(scenario_managers=["smCustomerAcquisition"],
        scenarios=["interactiveScenario"], 
        equations=['profit','customers'],
        return_df=True
        )
    df["advertisingSuccessPct"]=[setting["advertisingSuccessPct"]]*len(df.index)
    df["referralFreeMonths"]=[setting["referralFreeMonths"]]*len(df.index)
    df["referralProgramAdoptionPct"]=[setting["referralProgramAdoptionPct"]]*len(df.index)
    df["referrals"]=[setting["referrals"]]*len(df.index)
    df["time"]=df.index
    interactive_dfs += [df]
interactive_tab = pd.concat(interactive_dfs, ignore_index=True, sort=False) 

In [33]:
interactive_tab.to_excel('/Users/olivergrasl/Desktop/test.xlsx', sheet_name='interactive')

In [3]:
from IPython.display import IFrame       
IFrame('https://app.powerbi.com/view?r=eyJrIjoiNDFlMTA2ZGItYmFkOC00Mjg3LTljODYtYzRkYzY2YmFmM2E4IiwidCI6ImZjNWRmZDc1LTQ1NzktNGQ3MC05YTE3LTk0MmQ0OWMwMTc2ZiIsImMiOjl9', width=933, height=700)